<h2>Importing Libraries</h2>
<h6>
    <ul>
        <li><a href="https://selenium-python.readthedocs.io/index.html">Selenium (pip install selenium)</a></li>
        <li><a href="https://pypi.org/project/webdriver-manager/">Webdriver Manager (pip install webdriver-manager)</a></li>
        <li><a href="https://pypi.org/project/requests/">Requests (python -m pip install requests)</a> [UNUSED]</li>
        <li><a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">BeautifulSoup (pip install beautifulsoup4)</a></li>
        <li><a href="https://pandas.pydata.org/docs/">Pandas (pip install pandas)</a</li>
</h6>


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

<h2>Constants</h2>


In [2]:
#============================Unused============================#
HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

<h2>Functions</h2>

<h3>Set up functions</h3>

In [3]:
def set_up():
    what=input('O quê: ')
    what=what.replace(' ','+')
    where=input('Onde: ')
    where=where.replace(' ','+')
    range_=int(input('Alcance: '))
    return what, where, range_

<h3>Scrapping Module</h3>

In [4]:
def build_url(what,where,page):
    url=f'https://br.indeed.com/jobs?q={what}&l={where}&start={page}'
    return url

In [6]:
#============================Unused============================#
def get_current_url(driver, url, job_title, location=None):
    #driver = webdriver.Firefox()
    #driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(1)
    if location is not None:
        driver.find_element(By.XPATH,'//*[@id="text-input-where"]').send_keys(location)
        time.sleep(1)
    driver.find_element(By.XPATH,'/html/body/div').click()
    time.sleep(1)
    try:
        driver.find_element(By.XPATH,'//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element(By.XPATH,'//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url

    return current_url

In [7]:
def get_page_content(driver, url):
    #driver=webdriver.Firefox()
    time.sleep(1)
    driver.get(url)
    content=driver.page_source
    return content

In [ ]:
def get_jobs(content):
    soup=BeautifulSoup(content,'lxml')
    jobs_list=[]    
    for post in soup.select('.job_seen_beacon'):
        try:
            data={
                "Vaga":post.select('.jobTitle')[0].get_text().strip(),
                "Empresa":post.select('.companyName')[0].get_text().strip(),
                "Avaliação":post.select('.ratingNumber')[0].get_text().strip(),
                "Local":post.select('.companyLocation')[0].get_text().strip(),
                "Data":post.select('.date')[0].get_text().strip(),
                "Descrição":post.select('.job-snippet')[0].get_text().strip(),
                #"Link":f'br.indeed.com'+post.select('.jcs-JobTitle css-jspxzf eu4oa1w0').get('href').strip()
                "Link":f'br.indeed.com'+post.find('a',{'class':'jcs-JobTitle css-jspxzf eu4oa1w0'}).get('href').strip()    
            }
        except IndexError:
            continue          
        jobs_list.append(data)
    #dataframe = pd.DataFrame(jobs_list)
    
    return jobs_list

<h3>CSV Exporting Module</h3>

In [ ]:
def save_to_csv(jobs):
    file=open('jobs.csv',"w",encoding="utf-8")
    writer=csv.writer(file)
    writer.writerow(['Vaga','Empresa','Avaliação','Local','Data','Descrição','Link'])
    for job in jobs:
        writer.writerow(list(job.values()))

<h1>Execution</h1>

In [ ]:
what,where,range_=set_up()
driver = webdriver.Firefox()
'''
jobs=[]
for page in range(0,(range_*10),10):
    try:
        url=build_url(what,where,page)
        content=get_page_content(driver,url)
        jobs=get_jobs(content)
        print(f'&start={page}')
        print(f'found {len(jobs)} jobs')
        print(jobs)
        save_to_csv(jobs)
    except:
        break
#aderess=get_current_url(driver,'https://br.indeed.com/','Python','Rio de Janeiro, RJ')
'''
driver.quit()